# 'Maske' vor und nach Covid-19 - eine Untersuchung mit Word2Vec

#### Bedeutungen des Lemmas 'Maske'

1. künstliche feste Hülle vor dem Gesicht, Larve <br>
[bildlich] ... <br>
Deckmantel
2. Kostümierung, besonders das geschminkte Gesicht des Schauspielers
3. Träger von 1, Vermummte, Vermummter, Verkleidete, Verkleideter <br>
(Quelle: https://www.dwds.de/wb/Maske) <br> 
<br> 

#### Hypothese

Durch die Pandemie und die damit einhergehende Notwendigkeit von Seuchenschutzmaßnahmen gewinnt die Lesart 1), und dabei insbesondere die Bedeutung von Maske als Mund-Nasen-Schutz, an Popularität und verdrängt die anderen Bedeutungen. Die Bedeutung des Lemmas wandelt sich.

## Importe und Datenvorbereitung

In [1]:
import codecs
import matplotlib.pyplot as plt
import nltk
import numpy as np
import os
import pandas as pd
import re
import scipy
import spacy
import seaborn as sns

from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from gensim.models.phrases import Phraser, Phrases
from joblib import Parallel, delayed  
from nltk.corpus import stopwords
from scipy import spatial
from sklearn.manifold import TSNE
from tabulate import tabulate

In [2]:
nlp = spacy.load('de_core_news_md')
stopwords = stopwords.words('german')
tokenizer = nltk.data.load('tokenizers/punkt/german.pickle')

#### Korpus laden und teilen

In [3]:
# die Datei enthält alle Treffer des Lemmas 'maske' im ZDL-Regionalkorpus des DWDS 
# mit jeweils einem Satz Kontext davor und danach

df = pd.read_csv('../data/maske_1993-2021.csv', sep=',', encoding='utf-8')
df.head()

,Corpus,Date,Genre,Bibl,ContextBefore,Hit,ContextAfter
0,saar_regional,1993-02-01,Zeitung,"Saarbrücker Zeitung, 01.02.1993",NaN,Von der farbkräftigen Maske (Venezianischer Tr...,NaN
1,saar_regional,1993-02-06,Zeitung,"Saarbrücker Zeitung, 06.02.1993",[...] Opfer einer Serie von Vergewaltigungen z...,Der Mann mit der Maske ließ ab und flüchtete.,Der 23jährige Skinhead Markus L. hatte vor wen...
2,saar_regional,1993-02-10,Zeitung,"Saarbrücker Zeitung, 10.02.1993","[...] gibt 's ka Sünd "", worauf auch die Dekor...",Die schönsten Kostüme und Masken werden mit be...,"Zum Tanz spielt die ""happy music"" auf."
3,saar_regional,1993-02-10,Zeitung,"Saarbrücker Zeitung, 10.02.1993","[...]tiven, im besonderen an den Spätzündervor...","In bewährter Manier zeichneten verantwortlich,...",Als flitzender Butler fungierte Christian Klesen.
4,saar_regional,1993-02-11,Zeitung,"Saarbrücker Zeitung, 11.02.1993",[...]n und verließ fluchtartig die Schalterhalle.,Beim Verlassen der Poststelle in der Wadgasser...,NaN


In [4]:
# NaN durch Whitespace ersetzen

df = df.fillna(' ')

In [5]:
# Wörter, die mit [...] anfangen löschen

expression = '\[...]\w*'
df = df.replace(to_replace = expression, value = ' ', regex=True)

In [6]:
# bevor die Spalten verbunden werden: Whitespace einfügen, um ein Aneinanderkleben der Wörter zu verhindern

df['ContextBefore'] = df['ContextBefore'].astype(str) + ' '
df['ContextAfter'] = ' ' + df['ContextAfter'].astype(str) 

In [7]:
# Text zu einer Spalte verbinden

columns = ['ContextBefore', 'Hit', 'ContextAfter']

df['Text'] = df[columns].astype(str).sum(axis=1)
df.head()

,Corpus,Date,Genre,Bibl,ContextBefore,Hit,ContextAfter,Text
0,saar_regional,1993-02-01,Zeitung,"Saarbrücker Zeitung, 01.02.1993",,Von der farbkräftigen Maske (Venezianischer Tr...,,Von der farbkräftigen Maske (Venezianischer ...
1,saar_regional,1993-02-06,Zeitung,"Saarbrücker Zeitung, 06.02.1993",Opfer einer Serie von Vergewaltigungen zu we...,Der Mann mit der Maske ließ ab und flüchtete.,Der 23jährige Skinhead Markus L. hatte vor we...,Opfer einer Serie von Vergewaltigungen zu we...
2,saar_regional,1993-02-10,Zeitung,"Saarbrücker Zeitung, 10.02.1993","gibt 's ka Sünd "", worauf auch die Dekoratio...",Die schönsten Kostüme und Masken werden mit be...,"Zum Tanz spielt die ""happy music"" auf.","gibt 's ka Sünd "", worauf auch die Dekoratio..."
3,saar_regional,1993-02-10,Zeitung,"Saarbrücker Zeitung, 10.02.1993",", im besonderen an den Spätzündervorsitzenden...","In bewährter Manier zeichneten verantwortlich,...",Als flitzender Butler fungierte Christian Kle...,", im besonderen an den Spätzündervorsitzenden..."
4,saar_regional,1993-02-11,Zeitung,"Saarbrücker Zeitung, 11.02.1993",und verließ fluchtartig die Schalterhalle.,Beim Verlassen der Poststelle in der Wadgasser...,,und verließ fluchtartig die Schalterhalle. B...


In [8]:
df.shape

(154567, 8)

In [9]:
# Korpus in 2 Teilkorpora splitten: vor und nach Covid-19 (dem Ausbruch in Deutschland)
# hier wurde die Grenze zwischen Januar und Februar 2020 gezogen (es wird sich noch zeigen, inwiefern sich das bewährt)

df_before_corona = df.iloc[:75633,:]
df_after_corona = df.iloc[75634:,:]

In [10]:
df_after_corona.head()

,Corpus,Date,Genre,Bibl,ContextBefore,Hit,ContextAfter,Text
75634,sk_regional,2020-02-01,Zeitung,"Südkurier, 01.02.2020",", sondern beim Strählen in den Neudinger Gast...",Beim Strählen wird das Gesicht mit einer Maske...,"Die ersten Narreblättli-Ausgaben, die heute n...",", sondern beim Strählen in den Neudinger Gast..."
75635,sk_regional,2020-02-01,Zeitung,"Südkurier, 01.02.2020","Gründungsmitglieder, die sogenannten Gelbjac...",In ihrer Nachfolge gibt es seit 1993 die Gelbj...,Sie haben 33 Mitglieder.,"Gründungsmitglieder, die sogenannten Gelbjac..."
75636,sk_regional,2020-02-01,Zeitung,"Südkurier, 01.02.2020",Sie haben 33 Mitglieder.,Neue Cliquen: 1970 wurden die Erzgräber (33 Mi...,Die 1971 gegründete Gruppierung der Hexen wur...,Sie haben 33 Mitglieder. Neue Cliquen: 1970 wu...
75637,sk_regional,2020-02-01,Zeitung,"Südkurier, 01.02.2020",einzelnen Zünften wurden gesammelt und eine ...,Ein Häs mit den Konterfeis aller Masken und Ut...,"Es blieb nicht beim Häs für den Rathauschef, ...",einzelnen Zünften wurden gesammelt und eine ...
75638,sk_regional,2020-02-01,Zeitung,"Südkurier, 01.02.2020",,Die Zunftstube im Haus Mariagrün im Westen der...,,Die Zunftstube im Haus Mariagrün im Westen d...


#### Hilfsfunktionen zur Vorbereitung der Texte

In [11]:
def lemmatize_text_column(df, column):
    """
    transforms the Dataframe-column in a lemmatized string
    """
    text = ''
    for i in df[column]:
        doc = nlp(i)
        lemmas = ' '.join([x.lemma_ for x in doc])
        text = text + lemmas
    return text


def sentence_to_wordlist(raw:str):
    """
    cleans and tokenizes the sentences
    """
    text = re.sub('[^A-Za-z_äÄöÖüÜß]',' ', raw).split()
    filtered_text = [word for word in text if word not in stopwords]
    return filtered_text


def prepare_text(raw_text):
    """
    returns a list of tokenized sentences
    """
    raw_sentences = tokenizer.tokenize(str(raw_text).lower())    
    tokenized_sentences = Parallel(n_jobs=-1)(delayed(sentence_to_wordlist)(raw_sentence) for raw_sentence in raw_sentences)
    phrases = Phrases(tokenized_sentences)
    bigram = Phraser(phrases)
    sentences = list(bigram[tokenized_sentences])
    return sentences

#### Vorbereitung des ersten Texts (vor Covid-19)

In [12]:
text = lemmatize_text_column(df_before_corona, 'Text')
sentences = prepare_text(text)

# sentences ist eine Liste von tokenisierten Sätzen, zum Beispiel:
print(sentences[15])

['februar', 'veranstalten', 'turnverein', 'bildstock', 'heringswanderung']


#### Vorbereitung des zweiten Texts (nach Covid-19)

In [13]:
text2 = lemmatize_text_column(df_after_corona, 'Text')
sentences2 = prepare_text(text2)

print(sentences2[100])

['dpa', 'inkubationszeit', 'frist', 'möglich', 'ansteckung', 'krankheitsausbruch']


## Training von Word2Vec auf den Text vor Covid-19

In [14]:
# Paramter setzen
workers = 4                      # Use these many worker threads to train the model (=faster training with multicore machines)
seed = 42                        # Seed for the random number generator

In [15]:
# Ordner anlegen zum Abspeichern von trainierten Modellen
if not os.path.exists('../trained_models'):
    os.makedirs('../trained_models')

In [16]:
# Training

w2v_bc = Word2Vec(sentences=sentences,                   
                 vector_size=300,          # Dimensionality of the word vectors
                 window=10,                # The maximum distance between the current and predicted word within a sentence
                 min_count=3,              # (int, optional) – The model ignores all words with total frequency lower than this
                 workers=workers, 
                 min_alpha=0.0001,         # Learning rate will linearly drop to min_alpha as training progresses
                 sg=1,                     # Training algorithm: skip-gram if sg=1, otherwise CBOW
                 seed=seed)

## Training von Word2Vec auf den Text nach Covid-19

In [17]:
w2v_ac = Word2Vec(sentences=sentences2,                   
                 vector_size=300,                
                 window=10,              
                 min_count=3,             
                 workers=workers, 
                 min_alpha=0.0001,                                                    
                 sg=1,                     
                 seed=seed)

In [18]:
# trainierte Modelle speichern
w2v_bc.save(os.path.join('../trained_models', 'w2v_bc_maske.model'))
w2v_ac.save(os.path.join('../trained_models', 'w2v_ac_maske.model'))

## Exploration und Vergleich der Embeddings

In [19]:
# trainierte Modelle laden
w2v_bc = Word2Vec.load(os.path.join('../trained_models', 'w2v_bc_maske.model'))
w2v_ac = Word2Vec.load(os.path.join('../trained_models', 'w2v_ac_maske.model'))

In [20]:
# ähnliche Wörter zu 'maske' vor Covid-19
w2v_bc.wv.most_similar(positive=['maske'], topn=25)

[('vollmasken', 0.5156558752059937),
 ('arbeitskleidung', 0.5113664269447327),
 ('autofahren', 0.5106855630874634),
 ('anheften', 0.5044383406639099),
 ('fast_grenzenlos', 0.5006393790245056),
 ('glitterrock', 0.49931174516677856),
 ('narr_steuer', 0.499008446931839),
 ('vorgerückt_stunde', 0.49759626388549805),
 ('mitternacht_maskenprämierung', 0.49652960896492004),
 ('spaßpäckchen', 0.4952717125415802),
 ('krummen', 0.4952276051044464),
 ('maskieren_ballbesucher', 0.4946368634700775),
 ('tierverkleidungen', 0.4939550459384918),
 ('faschingsrevolvern', 0.493613064289093),
 ('barfuß', 0.49353229999542236),
 ('handwerkszeug', 0.4918953776359558),
 ('kim_sandra', 0.4918002188205719),
 ('ergeht', 0.4912738800048828),
 ('prämierungskommission', 0.4909202754497528),
 ('motorradkutten', 0.490786075592041),
 ('quälerei_geben', 0.4901343882083893),
 ('hei_chang', 0.4901064932346344),
 ('anstrengen_bekennen', 0.4900094270706177),
 ('detailstrotzende', 0.4886721074581146),
 ('kleinkindergottesdi

In [21]:
# ähnliche Wörter zu 'maske' nach Covid-19
w2v_ac.wv.most_similar(positive=['maske'], topn=25)

[('hygienevorschriften_einhalten', 0.6479010581970215),
 ('abseits_sitzplatzes', 0.6431371569633484),
 ('dienstleistung', 0.6429378986358643),
 ('schulgebäudes', 0.6420866250991821),
 ('mitgeführt', 0.6408758759498596),
 ('all_teilnehmende', 0.6388278007507324),
 ('meter_garantieren', 0.6383103132247925),
 ('solange_sitzplatz', 0.6380438804626465),
 ('allgemein_coronaregeln', 0.637239396572113),
 ('erreichen_sitzplatzes', 0.6360487341880798),
 ('tagesaktueller', 0.6354535818099976),
 ('beim_schwimmen', 0.6343627572059631),
 ('liegewiese', 0.6339292526245117),
 ('gemeindegesang_erlauben', 0.6332786083221436),
 ('sonstig_grund', 0.6332252025604248),
 ('namensliste', 0.6326099038124084),
 ('besucher_innen', 0.6326025724411011),
 ('präparat', 0.6323673725128174),
 ('betriebsgelände', 0.6323316693305969),
 ('schnell_selbsttest', 0.6317701935768127),
 ('maskeder', 0.6312674283981323),
 ('verkehrsmittel', 0.6311208605766296),
 ('anlieferer', 0.6309932470321655),
 ('eigen_matte', 0.63084334135

In [22]:
w2v_ac.wv.most_similar(positive=['virus'], topn=10)

[('infektion', 0.7354362607002258),
 ('coronaviren', 0.7214908599853516),
 ('tödlich', 0.7195044159889221),
 ('coronavirus', 0.7194503545761108),
 ('corona_virus', 0.7181829214096069),
 ('aerosole', 0.708656370639801),
 ('verbreiten', 0.7079938650131226),
 ('verbreitung', 0.7076319456100464),
 ('tröpfcheninfektion', 0.7070354223251343),
 ('tröpfchen', 0.7018046379089355)]

## Ausrichtung der beiden Embedding-Modelle

In [23]:
# Quelle: https://gist.github.com/zhicongchen/9e23d5c3f1e5b1293b16133485cd17d8

def smart_procrustes_align_gensim(base_embed, other_embed, words=None):
    """
    Original script: https://gist.github.com/quadrismegistus/09a93e219a6ffc4f216fb85235535faf
    Updatet script: https://gist.github.com/zhicongchen/9e23d5c3f1e5b1293b16133485cd17d8
    Procrustes align two gensim models (to allow for comparison between same word across models).
    Code ported from HistWords <https://github.com/williamleif/histwords> by William Hamilton <wleif@stanford.edu>.
        
    First, intersect the vocabularies (see `intersection_align_gensim` documentation).
    Then do the alignment on the other_embed model.
    Replace the other_embed model's syn0 and syn0norm numpy matrices with the aligned version.
    Return other_embed.

    If `words` is set, intersect the two models' vocabulary with the vocabulary in words (see `intersection_align_gensim` documentation).
    """

    # patch by Richard So [https://twitter.com/richardjeanso) (thanks!) to update this code for new version of gensim
    # base_embed.init_sims(replace=True)
    # other_embed.init_sims(replace=True)

    # make sure vocabulary and indices are aligned
    in_base_embed, in_other_embed = intersection_align_gensim(base_embed, other_embed, words=words)
    
    # re-filling the normed vectors
    in_base_embed.wv.fill_norms(force=True)
    in_other_embed.wv.fill_norms(force=True)

    # get the (normalized) embedding matrices
    base_vecs = in_base_embed.wv.get_normed_vectors()
    other_vecs = in_other_embed.wv.get_normed_vectors()

    # just a matrix dot product with numpy
    m = other_vecs.T.dot(base_vecs) 
    # SVD method from numpy
    u, _, v = np.linalg.svd(m)
    # another matrix operation
    ortho = u.dot(v) 
    # Replace original array with modified one, i.e. multiplying the embedding matrix by "ortho"
    other_embed.wv.vectors = (other_embed.wv.vectors).dot(ortho)    
    
    return other_embed

def intersection_align_gensim(m1, m2, words=None):
    """
    Intersect two gensim models, m1 and m2.
    Only the shared vocabulary between them is kept.
    If 'words' is set (as list or set), then the vocabulary is intersected with this list as well.
    Indices are re-organized from 0..N in order of descending frequency (=sum of counts from both m1 and m2).
    These indices correspond to the new syn0 and syn0norm objects in both gensim models:
        -- so that Row 0 of m1.syn0 will be for the same word as Row 0 of m2.syn0
        -- you can find the index of any word on the .index2word list: model.index2word.index(word) => 2
    The .vocab dictionary is also updated for each model, preserving the count but updating the index.
    """

    # Get the vocab for each model
    vocab_m1 = set(m1.wv.index_to_key)
    vocab_m2 = set(m2.wv.index_to_key)

    # Find the common vocabulary
    common_vocab = vocab_m1 & vocab_m2
    if words: common_vocab &= set(words)

    # If no alignment necessary because vocab is identical...
    if not vocab_m1 - common_vocab and not vocab_m2 - common_vocab:
        return (m1,m2)

    # Otherwise sort by frequency (summed for both)
    common_vocab = list(common_vocab)
    common_vocab.sort(key=lambda w: m1.wv.get_vecattr(w, "count") + m2.wv.get_vecattr(w, "count"), reverse=True)
    # print(len(common_vocab))

    # Then for each model...
    for m in [m1, m2]:
        # Replace old syn0norm array with new one (with common vocab)
        indices = [m.wv.key_to_index[w] for w in common_vocab]
        old_arr = m.wv.vectors
        new_arr = np.array([old_arr[index] for index in indices])
        m.wv.vectors = new_arr

        # Replace old vocab dictionary with new one (with common vocab)
        # and old index2word with new one
        new_key_to_index = {}
        new_index_to_key = []
        for new_index, key in enumerate(common_vocab):
            new_key_to_index[key] = new_index
            new_index_to_key.append(key)
        m.wv.key_to_index = new_key_to_index
        m.wv.index_to_key = new_index_to_key
        
        print(len(m.wv.key_to_index), len(m.wv.vectors))
        
    return (m1,m2)

In [24]:
# Ausrichtung des "After-Corona-Modells" an das "Before-Corona-Modell"

w2v_ac_al = smart_procrustes_align_gensim(w2v_bc, w2v_ac)

17067 17067
17067 17067


In [25]:
# speichern
w2v_ac_al.save(os.path.join('../trained_models', 'w2v_ac_al_maske.model'))

In [26]:
# laden
w2v_ac_al = Word2Vec.load(os.path.join('../trained_models', 'w2v_ac_al_maske.model'))

In [27]:
# ähnliche Wörter zu 'maske' nach Covid-19
w2v_ac_al.wv.most_similar(positive=['maske'], topn=25)

[('mitgeführt', 0.6408758163452148),
 ('präparat', 0.6323674917221069),
 ('maskeder', 0.6312674880027771),
 ('schwimmbecken', 0.6305027604103088),
 ('liegen_bereiten', 0.6299520134925842),
 ('zuschauer_zulassen', 0.6297780871391296),
 ('gotteshauses', 0.6286482810974121),
 ('schwimmhalle', 0.6266847252845764),
 ('dazugehören', 0.6254522204399109),
 ('desinfizierung', 0.624439001083374),
 ('kinobesucher', 0.6243300437927246),
 ('umkleideraum', 0.6223716139793396),
 ('lang_strecke', 0.6200210452079773),
 ('sanitäranlagen', 0.6197080016136169),
 ('musikinstrument', 0.6191754937171936),
 ('abs', 0.6185153722763062),
 ('verwaltungsgebäude', 0.6183631420135498),
 ('örtlichkeit', 0.6178203225135803),
 ('grundstück', 0.6168320775032043),
 ('kursteilnehmer', 0.6168129444122314),
 ('wahren_bleiben', 0.6167773604393005),
 ('umkleidekabine', 0.6165366172790527),
 ('selfies', 0.6160262227058411),
 ('gesondert', 0.6150798201560974),
 ('konzertes', 0.6146277785301208)]

In [28]:
vector_maske_bc = w2v_bc.wv['maske']  
vector_maske_ac_al = w2v_ac_al.wv['maske'] 

cosine_maske = 1 - spatial.distance.cosine(vector_maske_bc, vector_maske_ac_al)
cosine_maske

0.8371921181678772

In [29]:
vector_deutschland_bc = w2v_bc.wv['deutschland']  
vector_deutschland_ac_al = w2v_ac_al.wv['deutschland'] 

cosine_deutschland = 1 - spatial.distance.cosine(vector_deutschland_bc, vector_deutschland_ac_al)
cosine_deutschland

0.7170157432556152

In [30]:
vector_nachricht_bc = w2v_bc.wv['nachricht']  
vector_nachricht_ac_al = w2v_ac_al.wv['nachricht'] 

cosine_nachricht = 1 - spatial.distance.cosine(vector_nachricht_bc, vector_nachricht_ac_al)
cosine_nachricht

0.7696213722229004

In [31]:
vector_schreiben_bc = w2v_bc.wv['schreiben']  
vector_schreiben_ac_al = w2v_ac_al.wv['schreiben'] 

cosine_schreiben = 1 - spatial.distance.cosine(vector_schreiben_bc, vector_schreiben_ac_al)
cosine_schreiben

0.6534724235534668

In [32]:
vector_sagen_bc = w2v_bc.wv['sagen']  
vector_sagen_ac_al = w2v_ac_al.wv['sagen'] 

cosine_sagen = 1 - spatial.distance.cosine(vector_sagen_bc, vector_sagen_ac_al)
cosine_sagen

0.8021664023399353

In [33]:
vector_mann_bc = w2v_bc.wv['mann']  
vector_mann_ac_al = w2v_ac_al.wv['mann'] 

cosine_mann = 1 - spatial.distance.cosine(vector_mann_bc, vector_mann_ac_al)
cosine_mann

0.7615640163421631

In [34]:
vector_frau_bc = w2v_bc.wv['frau']  
vector_frau_ac_al = w2v_ac_al.wv['frau'] 

cosine_frau = 1 - spatial.distance.cosine(vector_frau_bc, vector_frau_ac_al)
cosine_frau

0.743870198726654

In [35]:
vector_mensch_bc = w2v_bc.wv['mensch']  
vector_mensch_ac_al = w2v_ac_al.wv['mensch'] 

cosine_mensch = 1 - spatial.distance.cosine(vector_mensch_bc, vector_mensch_ac_al)
cosine_mensch

0.7617343664169312

In [36]:
vector_montag_bc = w2v_bc.wv['montag']  
vector_montag_ac_al = w2v_ac_al.wv['montag'] 

cosine_montag = 1 - spatial.distance.cosine(vector_montag_bc, vector_montag_ac_al)
cosine_montag

0.7040581107139587